# Erstellt einen Langzeitkalender und ordnet diesem Ferien- und Feiertage zu

- erstellt diese aus csv-Dateien für Feiertage / Ferien einen fortlaufenden Kalender
- Stand 18.03.2025


In [ ]:
import pandas as pd
import duckdb

In [ ]:
duck = duckdb.connect(database=':memory:', read_only=False)

In [ ]:
ft_pd = pd.read_csv('kalender/feiertag.csv', sep=';', parse_dates=['datum'])
ft_pd['art2'] = 'Feiertag'
ferien_pd = pd.read_csv('kalender/ferien.csv', sep=';')
zeugnis_pd = pd.read_csv('kalender/zeugnis.csv', sep=';', parse_dates=['datum'])

In [ ]:
ferien_pd

In [ ]:
#erstellt Serien aus den Kalender Abschnitten in der Feriendatei
ferien_series = ''
for index, rows in ferien_pd.iterrows():
    print(rows['start'], rows['ende'], rows['ferien'])
    #print(rows[0], rows[1], rows[2]) #Indezierung mit Integer läuft aus
    fer_temp = pd.Series(pd.date_range(rows['start'], rows['ende']), name='days').to_frame()
    fer_temp['ferien'] = rows['ferien']
    fer_temp['art1'] = 'Ferien'
    try:
        ferien_series = pd.concat([ferien_series, fer_temp])
    except:
        ferien_series = fer_temp
    

In [ ]:
all_days = pd.Series(pd.date_range('2018-01-01', '2026-12-31'), name='days').to_frame() # erstellt Langzeitkalender
all_days['tag_name'] = all_days['days'].dt.day_name(locale = 'de_DE.utf8')
all_days['tag_nr'] = all_days['days'].dt.dayofweek #Montag = 0

In [ ]:
all_days.tail()

In [ ]:
ft_pd.tail(10)

In [ ]:
ferien_series.tail(6)

In [ ]:
all_fer = all_days.merge(ferien_series, left_on='days', right_on='days', how='left', suffixes=('_left', '_right')).reset_index()

In [ ]:
all_fer.tail(15)

In [ ]:
all_fer_ft = all_fer.merge(ft_pd, left_on='days', right_on='datum', how='left', suffixes=('_left', '_right')).reset_index()

In [ ]:
all_fer_ft.tail(15)

### Verknüpfen mit Zeugnis 

In [ ]:
all_fer_ft_zeugnis = all_fer_ft.merge(zeugnis_pd, left_on='days', right_on='datum', how='left')

In [ ]:
all_fer_ft_zeugnis.drop(columns=['datum_y'], inplace=True)

In [ ]:
all_fer_ft_zeugnis

### Ersetzen der NaN -Werte für die spätere Auswertung

In [ ]:
ersetz_nan = {'art1': 'Schule', 'art2': 'kein Feiertag', 'zeugnis': '-'}
all_fer_ft_zeugnis = all_fer_ft_zeugnis.fillna(value=ersetz_nan)

In [ ]:
all_fer_ft_zeugnis.to_csv('kalender/all.csv', sep=';', columns=['days', 'tag_name','ferien','feiertag','art1','art2', 'zeugnis'], index=False)

In [ ]:
duck.register('all_fer_ft_zeugnis', all_fer_ft_zeugnis)

In [ ]:
duck.sql('SELECT * FROM all_fer_ft_zeugnis LIMIT 10')

In [ ]:
duck.sql("""
create or replace table all_tg1 as

         (select    
            days::date as datum, 
         tag_name, 
         tag_nr, 
         ferien,
         feiertag,
         case 
            when (tag_nr < 5)  and (feiertag is NULL) then 'Mo-Fr'
            when (tag_nr = 5) and (feiertag is NULL) then 'Sa'            
            else 'So/FT'         
         end as tg1,

        case 
            when (tag_nr < 5)  and (feiertag is NULL) and (ferien is NULL) then 'Mo-Fr S'
            when (tag_nr < 5)  and (feiertag is NULL) and (ferien is not NULL) then 'Mo-Fr F'
            when (tag_nr = 5) and (feiertag is NULL) then 'Sa'            
            else 'So/FT'         
         end as tg2,
         from all_fer_ft)         

""")

In [ ]:
duck.sql('SELECT * FROM all_tg1').df().to_csv('/var/www/all_tg.csv', sep=';', index=False)